In [1]:
#***************SIGN LANGUAGE TO REGIONAL LANGUAGE***************#
#author: @Divyesh Chhabra, @Manan Gandhi, @Bhavya Patel

In [1]:
import numpy as np

In [2]:
import cv2 as cv
from PIL import Image as im

In [3]:
import import_ipynb
from predict import predict

importing Jupyter notebook from predict.ipynb
importing Jupyter notebook from E:\Study\Projects\BTP\Sign Language to Regional Language\Dataset\ASL\data_classification.ipynb


C:\Users\Owner\anaconda3\envs\data science\lib\site-packages\nbformat\__init__.py:92: MissingIDFieldWarning: Code cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


Found 90560 images belonging to 39 classes.


In [4]:
startPt = tuple()
endPt = tuple()
drawing = bool
drawn = bool

In [5]:
def reset():
    global startPt,endPt,drawing,drawn
    
    startPt = (-1,-1)
    endPt = (-1,-1)
    drawing = False
    drawn = False

In [6]:
reset()

def drawROI(event,x,y,flags,param):
    global startPt,endPt,drawing,drawn
    
    if event == cv.EVENT_LBUTTONDOWN:
        startPt = endPt = (x,y)
        drawing = True
    if event == cv.EVENT_MOUSEMOVE and drawing:
        endPt = (x,y)
    if event == cv.EVENT_LBUTTONUP:
        drawn = True
        drawing = False

In [7]:
capture = cv.VideoCapture(0)

width = int(capture.get(cv.CAP_PROP_FRAME_WIDTH))
height = int(capture.get(cv.CAP_PROP_FRAME_HEIGHT))

windowWidth,windowHeight = width*2,height*2

cv.namedWindow("SIGN",cv.WINDOW_NORMAL)
cv.setMouseCallback("SIGN",drawROI)
cv.resizeWindow("SIGN",windowWidth,windowHeight)

while True:
    
    success,frame = capture.read()
    
    if not success:
        print("Unable to record Frames")
        break
        
    frameCopy = frame.copy()
    
    wK = cv.waitKey(1)
    
    if drawing or drawn:
        cv.rectangle(img=frameCopy,pt1=startPt,pt2=endPt,color=(255,0,255),thickness=2)
        if drawn:
            regionOfInterest = frameCopy[startPt[1]:endPt[1],startPt[0]:endPt[0]]
            cv.putText(img=frameCopy,text="SIGNS",org=(startPt[0],startPt[1]-5),
                       fontFace=cv.FONT_HERSHEY_SIMPLEX,fontScale=abs(endPt[0]-startPt[0])*0.005,
                       color=(0,255,255),thickness=4)
            
            sign = im.fromarray(regionOfInterest)
            sign = sign.resize((300,300))
            
            cv.putText(img=frameCopy,text=predict(sign),org=(50,50),
                       fontFace=cv.FONT_HERSHEY_SIMPLEX,fontScale=abs(endPt[0]-startPt[0])*0.005,
                       color=(0,255,255),thickness=4)
    
    cv.imshow("SIGN",frameCopy)
    
    if wK == ord('q') or wK == 27:
        break
    elif wK == ord('c'):
        reset()
        frameCopy = frame.copy()

capture.release()
cv.destroyAllWindows()